<h1 align = center>Union de data/Merge</h1>

In [1]:
import pandas as pd

In [19]:
games_data = pd.read_parquet('games.parquet')
items_data = pd.read_parquet('items.parquet')
reviews_data= pd.read_parquet('reviews.parquet')

<h2  align = center>Preparacion para la union de los dataframe <sub>/ Data preparation for merging DataFrames<sub></h2>

<h3  align = left> Dataframe item </h3>

In [21]:
# Columnas de importancia de la data de items
items_data = items_data[['user_id','item_id','playtime_forever']]

# Creacion de id único para la data de items
items_data['item_id'] = items_data['item_id'].astype(str)
items_data['id'] = items_data['user_id'] + items_data['item_id']

<h3  align = left> Dataframe reviews </h3>

In [22]:
# Cambio de tipo de dato a la columna 'item_id' del DataFrame 'games_data'
games_data['item_id'] = games_data['item_id'].astype(str)

# generar un identificador único en el DataFrame 'user_reviews_data'
reviews_data['item_id'] = reviews_data['item_id'].astype(str)
reviews_data['id'] = reviews_data['user_id'] + reviews_data['item_id']

<h3  align = left> Union <sub>/ Merge </h3>

In [23]:
# union de dataframes reviews_data y games_data por el 'item_id' y eliminacion de nulos
merged_df = reviews_data.merge(games_data, on='item_id', how='left')
merged_df.dropna(inplace=True)

# generar dataframe unico uniendo 'items_data' con 'merge_df' por el identificador unico 'id' de nombre 'steam_data'
steam_data = items_data.merge(merged_df, on='id')

# renombrando columnas del dataframe 'steam_data'
steam_data = steam_data.rename(columns={'user_id_x':'user_id'})
steam_data = steam_data.rename(columns={'item_id_x':'item_id'})

# eliminando columnas redundantes del dataframe 'steam_data'
steam_data.drop('user_id_y', axis='columns', inplace=True)
steam_data.drop('item_id_y', axis='columns', inplace=True)

# cambio de tipos de datos de algunas columnas
steam_data['release_year'] = steam_data['release_year'].astype(str)
steam_data['year'] = steam_data['year'].astype(str)
steam_data['playtime_forever'] = steam_data['playtime_forever'].astype(int)

# seleccionando columnas definitivas para el analisis
steam_data = steam_data[['id','user_id','item_id','title','genre','developer','release_year','price','recommend','year','sentiment_analysis','playtime_forever']]

In [24]:
steam_data

,id,user_id,item_id,title,genre,developer,release_year,price,recommend,year,sentiment_analysis,playtime_forever
0,7656119797098247922200,76561197970982479,22200,Zeno Clash,Action,ACE Team,2009,9.99,1,2011,2,271
1,7656119797098247922200,76561197970982479,22200,Zeno Clash,Indie,ACE Team,2009,9.99,1,2011,2,271
2,765611979709824791250,76561197970982479,1250,Killing Floor,Action,Tripwire Interactive,2009,19.99,1,2011,2,10006
3,js41637227300,js41637,227300,Euro Truck Simulator 2,Indie,SCS Software,2013,19.99,1,2013,2,551
4,js41637227300,js41637,227300,Euro Truck Simulator 2,Simulation,SCS Software,2013,19.99,1,2013,2,551
...,...,...,...,...,...,...,...,...,...,...,...,...
82408,76561198239215706730,76561198239215706,730,Counter-Strike: Global Offensive,Action,Valve,2012,14.99,1,2015,2,4659
82409,wayfeng730,wayfeng,730,Counter-Strike: Global Offensive,Action,Valve,2012,14.99,1,2015,1,42740
82410,76561198251004808253980,76561198251004808,253980,Enclave,RPG,Starbreeze,2003,4.99,1,2015,2,1098
82411,72947282842730,72947282842,730,Counter-Strike: Global Offensive,Action,Valve,2012,14.99,1,2015,0,33


<h4>Convercion de archivos a parquet<sub>/Converting files to Parquet format<sub></h4>

In [25]:
# exportando archivo separado por comas de nombre 'data_steam'
steam_data.to_parquet('data_steam.parquet', index=False)